<a href="https://colab.research.google.com/github/eyamilabraham/convolutional-neural-networks-numbers-mode-3-RNC-with-data-increase-and-dropout/blob/main/convolutional_neural_networks_numbers_mode_3_RNC_with_data_increase_and_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras.utils import to_categorical

#Downloading MNIST dataset (handwritten numbers, labeled)
#Data Augmentation from Keras
(X_entrenamiento, Y_entrenamiento), (X_pruebas, Y_pruebas) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#Put each data in right way(1, 28, 28, 1)
X_training = X_training.reshape(X_training.shape[0], 28, 28, 1)
X_testing = X_testing.reshape(X_testing.shape[0], 28, 28, 1)

#Making 'one-hot encoding' from results (e.g. instead of having only one neuron as a result, I will have 10 where only the correct result will be 1 and the rest will be 0.)
Y_training = to_categorical(Y_training)
Y_testing = to_categorical(Y_testing)

#Convert to float and normalize the network that it could learn faster
X_training = X_training.astype('float32') / 255
X_testing = X_testing.astype('float32') / 255

NameError: ignored

In [ ]:
#Code to display pictures from dataset
import matplotlib.pyplot as plt
raws = 2
columns = 8
num = raws*columns
pictures = X_training[0:num]
labels = Y_training[0:num]
fig, axes = plt.subplots(raws, columns, figsize=(1.5*columns,2*raws))
for i in range(num):
     ax = axes[i//columns, i%columns]
     ax.imshow(pictures[i].reshape(28,28), cmap='gray_r')
     ax.set_title('Label: {}'.format(np.argmax(labels[i])))
plt.tight_layout()
plt.show()

In [ ]:
#Increase of data
#Variables to control transformations to increase of data
#Deploying ImageDataGenerator from keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

rotation_range = 30
mov_width = 0.25
mov_height = 0.25
#inclination_rangen=15 
approach_range=[0.5,1.5]

datagen = ImageDataGenerator(
    rotation_range = rotation_range,
    width_shift_range = mov_width,
    height_shift_range = mov_height,
    zoom_range=approach_range,
    #shear_range=inclination_range
)

datagen.fit(X_training)

In [ ]:
#Code to display pictures from dataset before and after of one of transformations
raws = 4
columns = 8
num = raws*columns
print('BEFORE:\n')
fig1, axes1 = plt.subplots(raws, columns, figsize=(1.5*columns,2*raws))
for i in range(num):
     ax = axes1[i//columns, i%columns]
     ax.imshow(X_training[i].reshape(28,28), cmap='gray_r')
     ax.set_title('Label: {}'.format(np.argmax(Y_training[i])))
plt.tight_layout()
plt.show()
print('AFTER:\n')
fig2, axes2 = plt.subplots(raws, columns, figsize=(1.5*columns,2*raws))
for X, Y in datagen.flow(X_entrenamiento,Y_entrenamiento.reshape(Y_entrenamiento.shape[0], 10),batch_size=num,shuffle=False):
     for i in range(0, num):
          ax = axes2[i//columnas, i%columnas]
          ax.imshow(X[i].reshape(28,28), cmap='gray_r')
          ax.set_title('Label: {}'.format(int(np.argmax(Y[i]))))
     break
plt.tight_layout()
plt.show()

In [ ]:
#Modelo!
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax")
])

#Compilación
modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Los datos para entrenar saldran del datagen, de manera que sean generados con las transformaciones que indicamos
data_gen_entrenamiento = datagen.flow(X_entrenamiento, Y_entrenamiento, batch_size=32)

In [ ]:
TAMANO_LOTE = 32

#Entrenar la red. Toma un buen rato! Ve por un café ;)
#Oye suscribete al canal!
print("Entrenando modelo...");
epocas=60
history = modelo.fit(
    data_gen_entrenamiento,
    epochs=epocas,
    batch_size=TAMANO_LOTE,
    validation_data=(X_pruebas, Y_pruebas),
    steps_per_epoch=int(np.ceil(60000 / float(TAMANO_LOTE))),
    validation_steps=int(np.ceil(10000 / float(TAMANO_LOTE)))
)

print("Modelo entrenado!");

In [ ]:
#Exportar el modelo al explorador! (Mas detalle de esto en en mi video de exportacion: https://youtu.be/JpE4bYyRADI )
modelo.save('numeros_conv_ad_do.h5')

#Convertirlo a tensorflow.js
!pip install tensorflowjs

!mkdir carpeta_salida

!tensorflowjs_converter --input_format keras numeros_conv_ad_do.h5 carpeta_salida